In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

from pprint import pprint
import random
import asyncio as aio

In [2]:
def run(f, value=None):
    try:                       
        f.send(value)
    except StopIteration as e: 
        return e.value
    finally:
        return f

async def draw_graph(g):
    while True:
        await aio.sleep(3)
        nx.draw(g)

In [3]:
class SignalGraphBase(nx.MultiDiGraph):
    '''Signal Graph Base Type.'''
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._prng = random.Random()
    
    @staticmethod
    def _action_dict(key, data):
        if not data:
            return dict()
        return dict(key=key,weight=data['weight'], action=data['action'])
    
    @staticmethod
    def _weight_sorted(iterable):
        return sorted(iterable, key=lambda d: d['weight'], reverse=True)
    
    def edge_potential(self, source, target, key):
        return self.__class__._action_dict(
            key, 
            self.get_edge_data(source, target, key, default=None))
    
    def edge_potentials(self, source, target):
        actions = []
        for _, t, key, data in self.edges(source, keys=True, data=True):
            if target == t:
                actions.append(self.__class__._action_dict(key, data))
        return self.__class__._weight_sorted(actions)
    
    @property
    def random_generator(self):
        return self._prng
    
    @random_generator.setter
    def random_generator(self, value):
        self._prng = value
    
    def node_sample(self, f, *, sample_size=None):
        if sample_size is not None:    
            sample_size = min(self.order(), sample_size) 
        else:
            sample_size = self.order()
        yield from map(f, ((n, self.nodes[n]) 
                           for n in self._prng.sample(self.nodes, sample_size)))
    
    def random_node(self, count=1):
        while True:
            yield from self._prng.sample(self.nodes, count)
            

In [4]:
class SourceGraph(nx.MultiDiGraph):
    '''Source Graph Type.
    
    This is a graph which can receive signals from
    an exterior source and update its nodes and edges
    from them.
    '''
    
    def __init__(self, *args, sources={}, source_key=id, **kwargs):
        super().__init__(*args, **kwargs)
        self.sources = sources
        self._source_key_mapping = source_key
    
    @property
    def source_key_mapping(self):
        return self._source_key_mapping
    
    @source_key_mapping.setter
    def source_key_mapping(self, mapping):
        self._source_key_mapping = mapping
        new_sources = {}
        for _, v in self.sources.items():
            new_sources[self._source_key(v)] = v
        self.sources = new_sources
    
    def add_source(self, source):
        self.sources[self._source_key_mapping(source)] = source
    
    def add_sources(self, source, *sources):
        add_source(source)
        for s in sources:
            add_source(s)
    
    def remove_source(self, source_id):
        del self.sources[source_id]
    
    def remove_sources(self, source_id, *source_ids):
        remove_source(source_id)
        for s in source_ids:
            remove_source(s)
    
    def clear_sources(self):
        self.sources = {}


In [9]:
g1 = SignalGraphBase()

async def action1():
    print('action1!')

async def action2():
    print('action2!')
    
async def action3():
    print('action3!')
    
async def sample_action(weight, source, target, source_weight, target_weight):
    while True:
        st_ratio = source_weight / target_weight
        source['memory'].append(weight + target_weight)
        target['memory'] = [unit * st_ratio for unit in source['memory']]
        await aio.sleep(0)

g1.add_node(1, weight=0.3, state=dict(memory=[]))
g1.add_node(2, weight=0.1, state=dict(memory=[]))
g1.add_edge(1, 2, key=1, weight=0.3, action=action1)
g1.add_edge(1, 2, key=1, weight=0.3, action=action1)
g1.add_edge(1, 2, key=2, weight=0.6, action=action2)
g1.add_edge(1, 2, key=3, weight=0.3, action=action3)
g1.add_edge(1, 1, key='a', weight=1, action=action1)
pprint(g1.edge_potentials(1, 2))
for n in g1.node_sample(print):
    pass

draw_graph(g1)

[{'action': <function action2 at 0x119427400>, 'key': 2, 'weight': 0.6},
 {'action': <function action1 at 0x119427730>, 'key': 1, 'weight': 0.3},
 {'action': <function action3 at 0x119427488>, 'key': 3, 'weight': 0.3}]
(1, {'weight': 0.3, 'state': {'memory': []}})
(2, {'weight': 0.1, 'state': {'memory': []}})


<coroutine object draw_graph at 0x119424d48>